In [1]:
import os
import numpy as np
import theano
import theano.tensor as T
import lasagne

Using gpu device 0: GeForce GT 645M (CNMeM is disabled, cuDNN 5105)
/usr/local/lib/python2.7/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
from lasagne.layers import InputLayer, DenseLayer, NonlinearityLayer
from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer
from lasagne.layers import Pool2DLayer as PoolLayer
from lasagne.nonlinearities import softmax

def build_model():
    net = {}
    net['input'] = InputLayer((None, 3, 224, 224))
    net['conv1_1'] = ConvLayer(net['input'], 64, 3, pad=1)
    net['conv1_2'] = ConvLayer(net['conv1_1'], 64, 3, pad=1)
    net['pool1'] = PoolLayer(net['conv1_2'], 2)
    net['conv2_1'] = ConvLayer(net['pool1'], 128, 3, pad=1)
    net['conv2_2'] = ConvLayer(net['conv2_1'], 128, 3, pad=1)
    net['pool2'] = PoolLayer(net['conv2_2'], 2)
    net['conv3_1'] = ConvLayer(net['pool2'], 256, 3, pad=1)
    net['conv3_2'] = ConvLayer(net['conv3_1'], 256, 3, pad=1)
    net['conv3_3'] = ConvLayer(net['conv3_2'], 256, 3, pad=1)
    net['pool3'] = PoolLayer(net['conv3_3'], 2)
    net['conv4_1'] = ConvLayer(net['pool3'], 512, 3, pad=1)
    net['conv4_2'] = ConvLayer(net['conv4_1'], 512, 3, pad=1)
    net['conv4_3'] = ConvLayer(net['conv4_2'], 512, 3, pad=1)
    net['pool4'] = PoolLayer(net['conv4_3'], 2)
    net['conv5_1'] = ConvLayer(net['pool4'], 512, 3, pad=1)
    net['conv5_2'] = ConvLayer(net['conv5_1'], 512, 3, pad=1)
    net['conv5_3'] = ConvLayer(net['conv5_2'], 512, 3, pad=1)
    net['pool5'] = PoolLayer(net['conv5_3'], 2)
    net['fc6'] = DenseLayer(net['pool5'], num_units=4096)
    net['fc7'] = DenseLayer(net['fc6'], num_units=4096)
    net['fc8'] = DenseLayer(net['fc7'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc8'], softmax)

    return net

In [ ]:
import cPickle
d = cPickle.load(open('vgg16_2.pkl'))
net = build_model()
lasagne.layers.set_all_param_values(net['prob'], d['param values'])

In [ ]:
del d

In [5]:
output_layer = DenseLayer(net['fc7'], num_units=2, nonlinearity=softmax)

X_sym = T.tensor4()
y_sym = T.ivector()

prediction = lasagne.layers.get_output(output_layer, X_sym)
loss = lasagne.objectives.categorical_crossentropy(prediction, y_sym)
loss = loss.mean()

acc = T.mean(T.eq(T.argmax(prediction, axis=1), y_sym),
                      dtype=theano.config.floatX)

params = lasagne.layers.get_all_params(output_layer, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.0001, momentum=0.9)

In [6]:
train_fn = theano.function([X_sym, y_sym], loss, updates=updates)
val_fn = theano.function([X_sym, y_sym], [loss, acc])
pred_fn = theano.function([X_sym], prediction)

In [9]:
import tables
hdf5_path = "trainingData.hdf5"
file = tables.open_file(hdf5_path, mode='r')
iteration = 0
TOTAL_NO_OF_IMAGES = 7020
for i in xrange(0, TOTAL_NO_OF_IMAGES/3, 270):
    images = np.concatenate((file.root.images[i:i+270], 
                            file.root.images[TOTAL_NO_OF_IMAGES-i-270:TOTAL_NO_OF_IMAGES-i]))
    labels = np.concatenate((file.root.labels[i:i+270], 
                            file.root.labels[TOTAL_NO_OF_IMAGES-i-270:TOTAL_NO_OF_IMAGES-i]))
    BATCH_SIZE = 1
    for epoch in range(5):
        for i in xrange(len(images)):
            loss = train_fn(images[i: i+BATCH_SIZE], labels[i: i+BATCH_SIZE])
            print 'iteration {0} complete'.format(iteration)
            iteration += 1
        print "Epoch {0} complete.".format(epoch)

file.close()

iteration 0 complete
iteration 1 complete
iteration 2 complete
iteration 3 complete
iteration 4 complete
iteration 5 complete
iteration 6 complete
iteration 7 complete
iteration 8 complete
iteration 9 complete
iteration 10 complete
iteration 11 complete
iteration 12 complete
iteration 13 complete
iteration 14 complete
iteration 15 complete
iteration 16 complete
iteration 17 complete
iteration 18 complete
iteration 19 complete
iteration 20 complete
iteration 21 complete
iteration 22 complete
iteration 23 complete
iteration 24 complete
iteration 25 complete
iteration 26 complete
iteration 27 complete
iteration 28 complete
iteration 29 complete
iteration 30 complete
iteration 31 complete
iteration 32 complete
iteration 33 complete
iteration 34 complete
iteration 35 complete
iteration 36 complete
iteration 37 complete
iteration 38 complete
iteration 39 complete
iteration 40 complete
iteration 41 complete
iteration 42 complete
iteration 43 complete
iteration 44 complete
iteration 45 complet

In [10]:
model = lasagne.layers.get_all_param_values(output_layer)

In [11]:
with open('vgg160n500weights.pkl','wb') as f:
    cPickle.dump(model, f, protocol = 2)

In [8]:
import cPickle

In [9]:
with open('vgg160n500weights.pkl','rb') as f:
    model = cPickle.load(f)

In [5]:
net = build_model()
output_layer = DenseLayer(net['fc7'], num_units=2, nonlinearity=softmax)

In [10]:
lasagne.layers.set_all_param_values(output_layer, model)

#### I had forgotten to include the final dense layer weights while dumping the trained network, so this part (only) of the network needs to be retrained.

In [7]:
for key,val in net.iteritems():
    if 'pool' in key or (key=='prob' or key=='input'):
        pass
    else:
        net[key].params[net[key].W].remove("trainable")
        net[key].params[net[key].b].remove("trainable")

In [8]:
X_sym = T.tensor4()
y_sym = T.ivector()

prediction = lasagne.layers.get_output(output_layer, X_sym)
loss = lasagne.objectives.categorical_crossentropy(prediction, y_sym)
loss = loss.mean()

acc = T.mean(T.eq(T.argmax(prediction, axis=1), y_sym),
                      dtype=theano.config.floatX)

params = lasagne.layers.get_all_params(output_layer, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.0001, momentum=0.9)

In [ ]:
train_fn = theano.function([X_sym, y_sym], loss, updates=updates)
val_fn = theano.function([X_sym, y_sym], [loss, acc])
pred_fn = theano.function([X_sym], prediction)

import tables
hdf5_path = "trainingData.hdf5"
file = tables.open_file(hdf5_path, mode='r')
iteration = 0
TOTAL_NO_OF_IMAGES = 7020
for i in xrange(0, TOTAL_NO_OF_IMAGES/3, 270):
    images = np.concatenate((file.root.images[i:i+270], 
                            file.root.images[TOTAL_NO_OF_IMAGES-i-270:TOTAL_NO_OF_IMAGES-i]))
    labels = np.concatenate((file.root.labels[i:i+270], 
                            file.root.labels[TOTAL_NO_OF_IMAGES-i-270:TOTAL_NO_OF_IMAGES-i]))
    BATCH_SIZE = 1
    for epoch in range(5):
        for i in xrange(len(images)):
            loss = train_fn(images[i: i+BATCH_SIZE], labels[i: i+BATCH_SIZE])
            print 'iteration {0} complete'.format(iteration)
            iteration += 1
        print "Epoch {0} complete.".format(epoch)
        
model = lasagne.layers.get_all_param_values(output_layer)
with open('vgg160n500weights.pkl','wb') as f:
    cPickle.dump(model, f, protocol = 2)

iteration 0 complete
iteration 1 complete
iteration 2 complete
iteration 3 complete
iteration 4 complete
iteration 5 complete
iteration 6 complete
iteration 7 complete
iteration 8 complete
iteration 9 complete
iteration 10 complete
iteration 11 complete
iteration 12 complete
iteration 13 complete
iteration 14 complete
iteration 15 complete
iteration 16 complete
iteration 17 complete
iteration 18 complete
iteration 19 complete
iteration 20 complete
iteration 21 complete
iteration 22 complete
iteration 23 complete
iteration 24 complete
iteration 25 complete
iteration 26 complete
iteration 27 complete
iteration 28 complete
iteration 29 complete
iteration 30 complete
iteration 31 complete
iteration 32 complete
iteration 33 complete
iteration 34 complete
iteration 35 complete
iteration 36 complete
iteration 37 complete
iteration 38 complete
iteration 39 complete
iteration 40 complete
iteration 41 complete
iteration 42 complete
iteration 43 complete
iteration 44 complete
iteration 45 complet

### TESTING THE CNN

In [11]:
import tables
hdf5_path = "trainingData.hdf5"
file = tables.open_file(hdf5_path, mode='r')

NO_OF_TEST_IMAGES = 7020/3 # 1/3 of total dataset for testing

# the test dataset
X_test = file.root.images[NO_OF_TEST_IMAGES:2*NO_OF_TEST_IMAGES] 
y_test = file.root.labels[NO_OF_TEST_IMAGES:2*NO_OF_TEST_IMAGES]

BATCH_SIZE = 1
cummLoss = 0
cummAcc = 0
for i in xrange(len(X_test)):
    loss, acc = val_fn(X_test[i:i+BATCH_SIZE], y_test[i:i+BATCH_SIZE])
    cummLoss += loss
    cummAcc += acc
    print 'Image {0} classified'.format(i+1)

loss = cummLoss/len(X_test)
acc = cummAcc/len(y_test)
print 'Accuracy = ', acc

Image 1 classified
Image 2 classified
Image 3 classified
Image 4 classified
Image 5 classified
Image 6 classified
Image 7 classified
Image 8 classified
Image 9 classified
Image 10 classified
Image 11 classified
Image 12 classified
Image 13 classified
Image 14 classified
Image 15 classified
Image 16 classified
Image 17 classified
Image 18 classified
Image 19 classified
Image 20 classified
Image 21 classified
Image 22 classified
Image 23 classified
Image 24 classified
Image 25 classified
Image 26 classified
Image 27 classified
Image 28 classified
Image 29 classified
Image 30 classified
Image 31 classified
Image 32 classified
Image 33 classified
Image 34 classified
Image 35 classified
Image 36 classified
Image 37 classified
Image 38 classified
Image 39 classified
Image 40 classified
Image 41 classified
Image 42 classified
Image 43 classified
Image 44 classified
Image 45 classified
Image 46 classified
Image 47 classified
Image 48 classified
Image 49 classified
Image 50 classified
Image 51 

In [12]:
file.close()